In [1]:
import os
os.chdir("..")

# Préparation 

## Imports

In [2]:
import pandas as pd

pd.options.display.max_columns = 100

In [3]:
from src.utils import get_all_schema_path
from tableschema import Schema

## Lecture et mise en forme

In [4]:
df = pd.read_csv("notebooks/Envoi_Dico Alimentation/Table_Variable_Modele_181214170701.csv", 
                 encoding="latin1", sep=';')
df = df.drop(columns=['Chemin', 'Unnamed: 21', 'Unnamed: 22'])

In [5]:
df.columns = ['produit', 
              'name_table', 
              'title_table', 
              'name', 
              'description',
              'famille_concept', 
              'length', 
              'type', 
              'nomenclature', 
              'elementaire_ou_calculee', 
              'classe_donnee', 
              'regle_gestion', 
              'observation_variable',
              'sensible_ou_medicale', 
              'type_objet', 
              'mode_acces',
              'observation_produit_variable', 
              'liaison', 
              'observation_cumul', 
              'referentiel']

In [6]:
df.head(2)

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
0,IR_BEN_R,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,ASS_NIR_ANO,Matricule anonymisé de l'ouvreur de droits,Bénéficiaires,17,Caractère,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAS Guide,NaN,IR_BEN_RASS_NIR_ANO,\n0,NaN
1,IR_BEN_R,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_ANO,Identifiant bénéficiaire anonymisé,Bénéficiaires,18,Varchar,NaN,Calculée,NaN,NaN,NaN,NaN,NaN,SAS Guide,NaN,IR_BEN_RBEN_IDT_ANO,\n0,NaN


In [7]:
df.duplicated().sum()

176

In [8]:
df = df.drop_duplicates()

## Signification variable Liaison
combinaison du nom de table et variable, ou produit et …

In [9]:
mask = ((df.name_table + df.name != df.liaison) & 
        (df.produit + df.name != df.liaison) & 
        (df.produit + df.name_table != df.liaison)
       )
df[mask].liaison.str[:4].value_counts()

Series([], Name: liaison, dtype: int64)

## Nomenclatures

In [10]:
from src.utils import get_all_nomenclatures_csv_schema_path

nomenclatures_set = set(csv_path.split("/")[-1][:-4] for csv_path, _ in get_all_nomenclatures_csv_schema_path())

## Standardize values

In [11]:
def map_with_mapping(s):
    if s in mapping:
        return mapping[s]
    return s

### fillna et strip


In [12]:
df = df.fillna("")

In [13]:
for column in df.columns:
    df[column] = df[column].str.strip()

### name_table

In [14]:
df.name_table = df.name_table.str.upper()

### produit

In [15]:
mapping = {
    'CMDC': 'Causes de décès',
    'CARTO' : 'CARTOGRAPHIE_PATHOLOGIES',
    "IR_BEN_R" : "BENEFICIAIRE",
    "IR_IBA_R" : "BENEFICIAIRE",
    "IR_IMB_R" : "DCIR_DCIRS",
    "IR_ORC_R" : "DCIR_DCIRS",
    "IR_ETM_R" : "DCIR_DCIRS",
    "IR_ACS_R" : "DCIR_DCIRS",
    "IR_MAT_R" : "DCIR_DCIRS",
    "IR_MTT_R" : "DCIR_DCIRS"
}
df.produit = df.produit.map(lambda s: mapping[s] if s in mapping else s)
df.produit.value_counts()

CARTOGRAPHIE_PATHOLOGIES     711
DCIRS                        409
DCIR                         402
EGB                          390
EGBS                         377
Univers SNIREP               370
Univers BERF                 254
Univers AMOS-EXE-PRS         240
Univers DAMIR                168
Suivi actes de biologie      160
Univers AMOS-CDS             152
Univers AMOS_ODS_LPP         150
AMOS                         126
Univers AMOS-CCAM-Prest      125
Univers AMOS-CCAM-Associa    110
DAMIR                         77
BGNA                          75
Causes de décès               61
DCIR_DCIRS                    50
Univers UCD                   45
BENEFICIAIRE                  38
Suivi actes de pharmacie      31
Name: produit, dtype: int64

In [16]:
mapping = {
    'Référentiel médicalisé du SNIIRAM-SNDS': 'Référentiel médicalisé des bénéficiaires (ALD)',
    'Dépenses des 57 millions dassurés sélectionnés': "Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N",
    'Identifiants des 57 millions dassurés sélectionnés de lannée yyyy': "Table individus de la cartographie des pathologies pour l'année AAAA et l'algorithme N",
    'Caractérisation des 57 M dassurés pr 56 grp de pathologies': "Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N",    
}
df.title_table = df.title_table.map(lambda s: mapping[s] if s in mapping else s)

#### Carto

In [17]:
# => Vérification que toutes les tables sont identiques en fonction des années pour la carto

for table in ['DEP', 'IDE', 'IND']:
    for year1, year2 in [('2013', '2014'), ('2014', '2015')]:
        table1 = 'CT_{}_{}_G4'.format(table, year1)
        table2 = 'CT_{}_{}_G4'.format(table, year2)
        df1 = df[df.name_table == table1].reset_index(drop=True).drop(columns="name_table")
        df2 = df[df.name_table == table2].reset_index(drop=True).drop(columns="name_table")
        assert df1.equals(df2)

In [18]:
# Remise en forme du nom des tables
mask = df.name_table.str.startswith("CT_")
df.name_table[mask] = df[mask].name_table.str[:7] + 'AAAA_GN'
df = df.drop_duplicates()

#### name

In [19]:
df.name = df.name.str.rstrip("\xa0")

### referentiel

In [20]:
mapping = {
    'IR_IBA_R': 'IR_IBA_R',
    'référentiel des bénéficiaires du SNIIRAM IR_BEN_R' : 'IR_BEN_R',
    'DA_PRA_R': 'DA_PRA_R',
    'T_NMED - Référentiel des Médicaments': 'T_NMED',
    'RG_AR_DET_FT - DA_PRA_R': 'RG_AR_DET_FT - DA_PRA_R',
    'SNDA': 'SNDA',
    'Fichier National des Professionnels de Santé (FNPS)': 'FNPS',
}
df.referentiel = df.referentiel.map(map_with_mapping)
df.referentiel.value_counts()

                           3954
IR_IBA_R                     33
DA_PRA_R                     23
IR_BEN_R                     22
T_NMED                       10
RG_AR_DET_FT - DA_PRA_R       2
SNDA                          2
FNPS                          1
Name: referentiel, dtype: int64

### elementaire

In [21]:
mapping = {
    'Elémentaire': 'Élémentaire', 
    'Calculée': 'Calculée', 
    'Calculé': 'Calculée', 
    'Élémentaire': 'Élémentaire'
}
df.elementaire_ou_calculee = df.elementaire_ou_calculee.map(map_with_mapping)
df.elementaire_ou_calculee.value_counts()

               2896
Élémentaire     770
Calculée        381
Name: elementaire_ou_calculee, dtype: int64

### type

In [22]:
mapping = {
    "Numérique": "number",
    "Caractère": "string",
    "Date": "date",
    "numérique": "number",
    "caractère": "string",
    "VARCHAR2": "string",
    "Varchar": "string",
    "date": "date"
}
df.type = df.type.map(lambda s: mapping[s] if s in mapping else s)

df.type.value_counts()

string    1981
number    1772
date       278
            16
Name: type, dtype: int64

In [47]:
df.loc[df.name == "IND_RNM_BEN", "description"] = "Top RNIAM"
df.loc[df.name == "IND_RNM_BEN", "type"] = "string"
df.loc[df.name == "IND_RNM_BEN", "length"] = "1"

In [48]:
df[df.type == ""]

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
20,DCIR_DCIRS,IR_ACS_R,Table historique des affiliations à l'aide à l...,FAM_DRT_DSD,,,,,,,,,,,,SAS Guide,,IR_ACS_RFAM_DRT_DSD,0,
21,DCIR_DCIRS,IR_ACS_R,Table historique des affiliations à l'aide à l...,FAM_DRT_DSF,,,,,,,,,,,,SAS Guide,,IR_ACS_RFAM_DRT_DSF,0,
22,DCIR_DCIRS,IR_ACS_R,Table historique des affiliations à l'aide à l...,FAM_DRT_VRU,,,,,,,,,,,,SAS Guide,,IR_ACS_RFAM_DRT_VRU,,
45,DCIR_DCIRS,IR_ORC_R,Table historique des affiliations à un organis...,MLL_CTA_DSD,,,,,,,,,,,,SAS Guide,,IR_ORC_RMLL_CTA_DSD,0,
46,DCIR_DCIRS,IR_ORC_R,Table historique des affiliations à un organis...,MLL_CTA_DSF,,,,,,,,,,,,SAS Guide,,IR_ORC_RMLL_CTA_DSF,,
47,DCIR_DCIRS,IR_ETM_R,Table historique des exonérations du ticket mo...,BEN_EXO_DTD,,,,,,,,,,,,SAS Guide,,IR_ETM_RBEN_EXO_DTD,0,
48,DCIR_DCIRS,IR_ETM_R,Table historique des exonérations du ticket mo...,BEN_EXO_MTF,,,,,,,,,,,,SAS Guide,,IR_ETM_RBEN_EXO_MTF,0,
49,DCIR_DCIRS,IR_ETM_R,Table historique des exonérations du ticket mo...,BEN_EXO_DTF,,,,,,,,,,,,SAS Guide,,IR_ETM_RBEN_EXO_DTF,0,
50,DCIR_DCIRS,IR_MAT_R,Table historique de la maternité,BEN_GRS_DTE,,,,,,,,,,,,SAS Guide,,IR_MAT_RBEN_GRS_DTE,0,
51,DCIR_DCIRS,IR_MAT_R,Table historique de la maternité,BEN_DRA_AME,,,,,,,,,,,,SAS Guide,,IR_MAT_RBEN_DRA_AME,,


### lenght

In [63]:
df.loc[df.name == "ARO_THE_TAU", "length"] = "5"

In [23]:
mask = (df.length.str.contains('\.') | df.length.str.contains('\,'))
df[mask].head(2)

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
98,DCIRS,NS_PRS_F,Table des prestations,RGO_REM_TAU,Taux de remboursement (part Régime Obligatoire),Prestations,"5,2",number,,Élémentaire,,,,,,SAS Guide,,DCIRSRGO_REM_TAU,0,
106,DCIRS,NS_CAM_F,Table simplifiée des données de codage de la C...,CAM_ACT_PRU,Prix unitaire CCAM de l'acte médical,Prestations,"7,2",number,,,,,,,,SAS Guide,,DCIRSCAM_ACT_PRU,0,


In [24]:
df.length = df.length.str.replace('.', ',').str.strip(',')

### nomenclatures

DONE
- X ne pas prendre les nouveaux ir_pha_r dans er_pha_r et ns_pha_r
- X ER_TIP_F/NS_PRS_F : TIP_ACT_PRU -> pas IR_DTE_V (prix)
- X ER_TIP_F/NS_PRS_F : TIP_PUB_PRX -> pas IR_PRF_V (prix)
- X IR_IMB_R : INS_DTE -> pas IR_IMB_R
- X NS_BIO_F : BIO_ACT_QSN -> pas IR_BIO_R (quantité)
- X NS_PHA_F : PHA_ACT_PRU -> pas IR_PHA_R
- X dans dcir simplifié remplacer partout IR_ACT_V, IR_SPE_V par IR_SPA_D
- CT_IND_AAAA_GN : beaucoup d'erreurs a priori
- problème nomenclatures multiples
- BE_IDE_R à traiter (on l'a pas, la récupérer ?)

##### DONE



In [25]:
mask = df.nomenclature.str.contains(',')

df.loc[mask,["name_table", "name", "nomenclature"]]

,name_table,name,nomenclature
66,IR_IMB_R,MED_MTF_COD,"IR_CCI_V, IR_CIM_V"
327,NS_PRS_F,PSP_ACT_SPE,"IR_ACT_V, IR_SPE_V"
1460,EB_IMB_R,MED_MTF_COD,"IR_CCI_V, IR_CIM_V"
1699,ES_PRS_F,PSP_ACT_SPE,"IR_ACT_V, IR_SPE_V"
1864,KI_CCI_R,CAT_PCS_COD,"DS_PC1_V, DS_PC2_V"
1866,KI_CCI_R,DCD_CIM_COD,"IR_CCI_V, IR_CIM_V"
1900,KI_ECD_R,ECD_CIM_COD,"IR_CCI_V, IR_CIM_V"
2633,RG_AR_DET_FT_AAAAMM,PSP_ACT_SPE,"IR_ACT_V, IR_SPE_V"
2659,RG_AR_DET_FT,PSP_ACT_SPE,"IR_ACT_V, IR_SPE_V"


In [26]:
df.loc[mask, 'nomenclature'] = ''

In [27]:
mask = (df.nomenclature.isin(["BE_IDE_R"]))
df.loc[mask,["produit", "name_table", "name", "nomenclature"]]
df.loc[mask, 'nomenclature'] = ''

In [28]:
df.nomenclature = (df.nomenclature
                   .replace('IR_ACT_V, IR_SPE_V', "IR_SPA_D")
                   .replace('IR_IMB_R', '')
                  )

In [29]:
mask = df.name.isin(['PHA_ACT_PRU', 'BIO_ACT_QSN', 'INS_DTE', 'TIP_PUB_PRX', 'TIP_ACT_PRU'])
df.loc[mask, 'nomenclature'] = ''

In [30]:
mask = (df.nomenclature == "IR_PHA_R") & df.name_table.isin(['ER_PHA_F', 'NS_PHA_F'])
df.loc[mask, 'nomenclature'] = ''

In [31]:
mask = (df.name_table.isin(['CT_IND_AAAA_GN']) & df.nomenclature.isin(['IR_PRF_V', "IR_PHA_R", "IR_DTE_V"]))
df.loc[mask, 'nomenclature'] = ''

In [32]:
mask = (df.name_table.isin(['ER_DCT_F']) & df.nomenclature.isin(["IR_PMC_V"]))
df.loc[mask, 'nomenclature'] = ''

In [33]:
mask = df.name.isin(['INS_DTE'])
df[mask]

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
65,DCIR_DCIRS,IR_IMB_R,Référentiel médicalisé des bénéficiaires (ALD),INS_DTE,Date d'insertion,Technique,8,date,,Élémentaire,,,,,,SAS Guide,,IR_IMB_RINS_DTE,0,
1431,EGB,EB_IMB_R,Table des données médicalisées des bénéficiair...,INS_DTE,Date d'insertion,Technique,8,date,,Élémentaire,,,,,,SAS Guide,Dans la table du référentiel médicalisé IR_IMB...,EGBINS_DTE,Dans la table du référentiel médicalisé IR_IMB...,
1432,EGB,EB_INB_F,référentiel des bénéficiaires de l'EGB,INS_DTE,Date d'insertion,Technique,8,date,,Élémentaire,,,,,,SAS Guide,Dans la table du référentiel médicalisé IR_IMB...,EGBINS_DTE,Dans la table du référentiel médicalisé IR_IMB...,


### description

In [34]:
df[df.description.str.contains("EPK")].description

827     Indicateur Référentiel EPK
1216    Indicateur Référentiel EPK
Name: description, dtype: object

In [35]:
for abbrev, nom in [
    ("Cpt", "comptable"),
    ("Regul", "régulation"),
    ("Dcpte", "décompte"),
    ("Prs", "Prestation"),
    ("Pharma", "pharmacie"),
    ("EPK","Entente Préalable kinésithérapie")

]:
    df.description = df.description.str.replace(abbrev, nom)

In [36]:
mapping = {
    "anomalie": "",
}
df.description = df.description.map(lambda s: mapping[s] if s in mapping else s)

## Préparation produits

In [37]:
produit_a_garder = [
    'CARTOGRAPHIE_PATHOLOGIES',
 'DCIR',
 'DCIRS',
 'Causes de décès',
 'DCIR_DCIRS',
 'BENEFICIAIRE',
 ]

df = df[df.produit.isin(produit_a_garder)]

In [38]:
xdf = df[df.name_table == 'CT_IND_AAAA_GN']

In [42]:
df.head()

,produit,name_table,title_table,name,description,famille_concept,length,type,nomenclature,elementaire_ou_calculee,classe_donnee,regle_gestion,observation_variable,sensible_ou_medicale,type_objet,mode_acces,observation_produit_variable,liaison,observation_cumul,referentiel
0,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,ASS_NIR_ANO,Matricule anonymisé de l'ouvreur de droits,Bénéficiaires,17,string,,,,,,,,SAS Guide,,IR_BEN_RASS_NIR_ANO,0,
1,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_ANO,Identifiant bénéficiaire anonymisé,Bénéficiaires,18,string,,Calculée,,,,,,SAS Guide,,IR_BEN_RBEN_IDT_ANO,0,
2,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_IDT_TOP,Top identifiant bénéficiaire Anonymisé,Bénéficiaires,1,number,,Calculée,,,,,,SAS Guide,,IR_BEN_RBEN_IDT_TOP,0,
3,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_CDI_NIR,Code d'identification du NIR,Bénéficiaires,2,string,IR_NIR_V,,,,,,,SAS Guide,,IR_BEN_RBEN_CDI_NIR,0,
4,BENEFICIAIRE,IR_BEN_R,Référentiel des bénéficiaires du SNIIRAM-SNDS,BEN_DCD_AME,Année et mois de décès du bénéficiaire,Bénéficiaires,6,string,,,,,,,,SAS Guide,,IR_BEN_RBEN_DCD_AME,,


## Create descriptor

In [39]:
def get_field_descriptor(df, name, columns):
    return (df[df.name.str.upper() == name.upper()][columns]
            .to_dict(orient="records")[0])

In [62]:
for i, (produit, name_table) in df[["produit", 'name_table']].drop_duplicates().iterrows():
    # Restriction table
    sdf = df[(df.produit == produit) & (df.name_table == name_table)]
    
    assert sdf.title_table.nunique() == 1
    title = sdf.title_table.unique()[0]

   
    # Lecture schema
    schema_path = "schemas/{}/{}.json".format(produit, name_table)
    assert os.path.exists(schema_path)
    schema = Schema(schema_path)

    
    # MAJ table
    schema.descriptor.update({
        "name": name_table, 
        "title": title, 
        "produit": produit
    })
    
    
    # Différences d'ensembles de variables
    schema_names = set([name.upper() for name in schema.field_names]) 
    dico_names = set(sdf.name)
    set_schema_dico = schema_names - dico_names
    set_dico_schema = dico_names - schema_names
      

    if set_dico_schema:
        for field in set_dico_schema:
            print("- Variable absente dans le schéma et ajoutée automatiquement", field)
            columns_to_use = ['name', 'description', 'type', 'length', 'nomenclature']
            field_descriptor = get_field_descriptor(sdf, field, columns_to_use)
            field_descriptor["dateCreated"] = ""
            field_descriptor["dateDeleted"] = ""
            field_descriptor["dateMissing"] = []
            field_descriptor["nomenclature"] = "-" if field_descriptor["nomenclature"] == "" else field_descriptor["nomenclature"]
            print(field_descriptor)
            schema.add_field(field_descriptor)   
            
    
    # MAJ variables
    for field in schema.descriptor['fields']:
        print()
        print(produit, name_table, title) 

        name = field['name']
        if not name.upper() in dico_names:
            
            print('- Variable absente dans le dico cnam', name)
            continue
        
        columns_to_update = ['description', 'type', 'nomenclature', 'length']
        columns_to_update = ['nomenclature']
        field_descriptor = get_field_descriptor(sdf, name, columns_to_update)
        for key in columns_to_update:
            if field_descriptor[key] == "" or field_descriptor[key] is None:
                field_descriptor.pop(key)
        field.update(field_descriptor)
    
    
    try:
        schema.commit(strict=True)
    except Exception as e:
        print(e.errors)
        raise e
    
    schema.save(schema_path, ensure_ascii=False)


BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel des bénéficiaires du SNIIRAM-SNDS

BENEFICIAIRE IR_BEN_R Référentiel

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classification Commune des Actes Médicaux

DCIRS NS_CAM_F Table simplifiée des données de codage de la Classificatio

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de codage de pharmacie

DCIRS NS_PHA_F Table simplifiée des données de c

DCIRS NS_TRS_F Table simplifiée des données détaillées de transport

DCIRS NS_TRS_F Table simplifiée des données détaillées de transport

DCIRS NS_TRS_F Table simplifiée des données détaillées de transport

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités Communes de Dispensation

DCIRS NS_UCD_F Table simplifiée des données de codage des Unités 

- Variable absente dans le schéma et ajoutée automatiquement PRE_IND_PEL
{'name': 'PRE_IND_PEL', 'description': 'Indicateur Prescription en Ligne', 'type': 'string', 'length': '2', 'nomenclature': '-', 'dateCreated': '', 'dateDeleted': '', 'dateMissing': []}
- Variable absente dans le schéma et ajoutée automatiquement PRS_DIS_PRE
{'name': 'PRS_DIS_PRE', 'description': 'Dispositif de prévention', 'type': 'string', 'length': '2', 'nomenclature': '-', 'dateCreated': '', 'dateDeleted': '', 'dateMissing': []}
- Variable absente dans le schéma et ajoutée automatiquement CPL_REM_TAU
{'name': 'CPL_REM_TAU', 'description': "Taux de remboursement des compléments d'actes", 'type': 'number', 'length': '7,2', 'nomenclature': '-', 'dateCreated': '', 'dateDeleted': '', 'dateMissing': []}
- Variable absente dans le schéma et ajoutée automatiquement PRS_QTT_MAJ
{'name': 'PRS_QTT_MAJ', 'description': 'Quantité de majorations', 'type': 'number', 'length': '4', 'nomenclature': '-', 'dateCreated': '', 'dat

DCIR ER_TRS_F Table des données détaillées de transport
- Variable absente dans le dico cnam TRS_ACP_NOMC

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport
- Variable absente dans le dico cnam TRS_CDR_NOMC

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport
- Variable absente dans le dico cnam TRS_VEH_NUMC

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transport

DCIR ER_TRS_F Table des données détaillées de transp

DCIR ER_CAM_F Table des données de codage de la Classification Commune des Actes Médicaux

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables
- Variable absente dans le dico cnam DCT_ORD_NUM

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des données comptables

DCIR ER_CPT_F Table des 

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_DEP_AAAA_GN Table dépenses de la cartographie des pathologies pour l'année AAAA et 

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologies pour l'année AAAA et l'algorithme N

CARTOGRAPHIE_PATHOLOGIES CT_IND_AAAA_GN Table pathologies de la cartographie des pathologi

In [60]:
1

1